In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import utm

vancouver_path = "./vancouvercrime.csv"
crime_mapping_path = "./crime_type_mapping.csv"
df = pd.read_csv(vancouver_path)[:100]
crime_code_df = pd.read_csv(crime_mapping_path)

In [2]:
fact_df = pd.DataFrame(columns=['location_Key','crime_key', 'date_key','event_holiday_key', 'is_traffic','is_fatal','is_nighttime'])
location_df = pd.DataFrame(columns=['location_Key','city','location_name', 'neighborhood', 'latitude','longitude','crime_rate'])
crime_df= pd.DataFrame(columns=['crime_key','crime_category_name', 'crime_type_name','crime_details',
                                 'start_hour', 'start_minute',"start_second",
                                 'end_hour', 'end_minute',"end_second",
                                 'report_hour', 'report_minute',"report_second"])

In [3]:
crime_list = crime_code_df["Vancouver"].tolist()

In [4]:
for i, row in df.iterrows():
    if not(2015<=row['YEAR']<=2019):
        continue
        
    index = len(fact_df)
    
    # Van id starts from 10^7, Den id starts from 10^6
    location_id = index + 10000000
    city = "Vancouver"
    location = row["HUNDRED_BLOCK"]
    neighborhood = row["NEIGHBOURHOOD"]
    lat,lon = utm.to_latlon(row['X'],row['Y'],10,'U')
    crime_rate = 0
    location_df.loc[index] = [location_id,city,location,neighborhood,lat,lon,crime_rate]
    
    
    crime_id = index + 20000000
    
    crime_type_map = crime_code_df.iloc[crime_list.index(row['TYPE'])]
    crime_category_name = crime_type_map["corr Denver category"]
    crime_type_name = crime_type_map["corr Denver type"]
    crime_detail = crime_type_map["corr Denver detail"]
    
    na = np.nan
    year = row['YEAR']
    month = row['MONTH']
    day = row['DAY']
    hour = row['HOUR']
    minute = row['MINUTE']
    
    crime_df.loc[index] = [crime_id,crime_category_name,crime_type_name,crime_detail,hour,minute,0,na,na,na,na,na,na]
    
    str_month = str(month) if month>9 else '0'+str(month)
    str_day = str(day) if day>9 else '0'+str(day)
    date_id = int(str(year)+str_month+str_day)
    
    
    #TODO
    event_holiday_id = np.nan
    holiday=[]
    
    is_night = 0 if 7 <= hour <= 18 else 1
    
    fatal_keywords=['FATAL','DEAD','DEATH','KILL','HOMICIDE','MURDER']
    is_fatal = 0
    for keyword in fatal_keywords:
        for name in [crime_category_name,crime_type_name,crime_detail]:
            if keyword in name.upper():
                is_fatal = 1

    
    fact_df.loc[index] = [location_id,crime_id,date_id,event_holiday_id,crime_type_map["is_traffic"],crime_type_map['is_fatal'],is_night]

In [5]:
# fact_df convert float to int
for col in fact_df.columns:
    if col !="event_holiday_key":
        fact_df[[col]]=fact_df[[col]].astype(int)

In [6]:
crime_df.head()

,crime_key,crime_category_name,crime_type_name,crime_details,start_hour,start_minute,start_second,end_hour,end_minute,end_second,report_hour,report_minute,report_second
0,20000000,burglary,burglary-business-by-force,Burglary of a business with forced entry,2,6,0,NaN,NaN,NaN,NaN,NaN,NaN
1,20000001,burglary,burglary-business-by-force,Burglary of a business with forced entry,4,12,0,NaN,NaN,NaN,NaN,NaN,NaN
2,20000002,burglary,burglary-business-by-force,Burglary of a business with forced entry,20,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,20000003,burglary,burglary-business-by-force,Burglary of a business with forced entry,6,17,0,NaN,NaN,NaN,NaN,NaN,NaN
4,20000004,burglary,burglary-business-by-force,Burglary of a business with forced entry,20,53,0,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
location_df.head()

,location_Key,city,location_name,neighborhood,latitude,longitude,crime_rate
0,10000000,Vancouver,10XX SITKA SQ,Fairview,49.266678,-123.129029,0
1,10000001,Vancouver,10XX ALBERNI ST,West End,49.285255,-123.123649,0
2,10000002,Vancouver,10XX ALBERNI ST,West End,49.284981,-123.123053,0
3,10000003,Vancouver,10XX ALBERNI ST,West End,49.284794,-123.122946,0
4,10000004,Vancouver,10XX ALBERNI ST,West End,49.284715,-123.122824,0


In [8]:
fact_df.head()

,location_Key,crime_key,date_key,event_holiday_key,is_traffic,is_fatal,is_nighttime
0,10000000,20000000,20190307,NaN,0,0,1
1,10000001,20000001,20190827,NaN,0,0,1
2,10000002,20000002,20171114,NaN,0,0,1
3,10000003,20000003,20180302,NaN,0,0,1
4,10000004,20000004,20150204,NaN,0,0,1
